#Training YOLO on a Custom Dataset



#Extracting Annotations from JSON

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install mdai --quiet
!pip install pydicom==1.4.2 --quiet

import mdai

#Replace with your filename
JSON = '/content/gdrive/MyDrive/aimi_summer_internship/mdai_stanford_project_7YNdMkqb_annotations_2022-06-27-174815.json'
results = mdai.common_utils.json_to_dataframe(JSON)
#Annotations dataframe
annots_df = results['annotations']

In [ ]:
# some copy and paste from the md.ai documentation
import cv2
import shutil
import os

#Simplify table
columns_brief = ['id', 'StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID','labelName', 'data', 'annotationMode']
annots_df = annots_df[columns_brief]

#Box annotations
boxes = annots_df[annots_df.annotationMode == 'bbox']

#Extract box data
def extract_box_data(df):
    j = df.copy()
    j = j[(j.annotationMode == 'bbox') & (~j.data.isnull())]
    try:
        j['data'] = j['data'].apply(lambda x:json.loads(x.replace("'", "\"")))
    except:
        j['data']

    j['x'] = [d['x'] for _,d in j.data.iteritems()]
    j['y'] = [d['y'] for _,d in j.data.iteritems()]
    j['w'] = [d['width'] for _,d in j.data.iteritems()]
    j['h'] = [d['height'] for _,d in j.data.iteritems()]
    j = j.drop('data', axis=1)
    return j

boxes = extract_box_data(boxes)


In [ ]:
# organizes relevant data into a dictionary
data = {'ID': [],
        'label': [],
        'x': [],
        'y': [],
        'w': [],
        'h': []}

for i in boxes['SOPInstanceUID']:
  data['ID'].append(i)

for i in boxes['labelName']:
  data['label'].append(i)

for i in boxes['x']:
  data['x'].append(i)

for i in boxes['y']:
  data['y'].append(i)

for i in boxes['w']:
  data['w'].append(i)

for i in boxes['h']:
  data['h'].append(i)


In [ ]:
# accesses all files within the folder

path = "/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/jpgs"
files = os.listdir(path)

print(len(files))
print(files)

classes = {
    'Endotracheal Tube' : 0,
    'Central venous catheter' : 1,
    'Chest Tube': 2
}

# converts annotations into YOLO format
for i in range(len(data['ID'])):
 if "{}.jpg".format(data['ID'][i]) in files: # checks to see if the annotation
    f = open("/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/txts/{}.txt".format(data['ID'][i]), "a")
    classification = classes[data['label'][i]]
    jpeg = "/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/jpgs/{}.jpg".format(data['ID'][i])
    im = cv2.imread(jpeg, cv2.IMREAD_GRAYSCALE)
    h, w = im.shape
    x_center = (data['x'][i] + data['w'][i] / 2) / w
    y_center = (data['y'][i] + data['h'][i] / 2) / h
    width = data['w'][i] / w
    height = data['h'][i] / h
    f.write(str(classification) + ' ' + str(x_center) + ' ' + str(y_center) + ' ' + str(width) + ' ' + str(height) + "\n")

path = "/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/txts"
txt_files = os.listdir(path)
print(len(txt_files))

2100
['1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736003950.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736004116.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736004174.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736004097.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736004018.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736004022.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736004173.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736004098.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736004169.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736004026.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736004119.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736004136.jpg', '1.3.6.1.4.1.1337.100.1.1.111165684411017669021768385720736003995.jpg', '1.3.6.1.4.1.1337.100.1.1.11116568441101766902176838572073

In [ ]:
for i in files:
  if (i[:-4] +'.txt') not in txt_files:
    f = open("/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/txts/{}.txt".format(i[:-4]), "w")
    print(i)
path = "/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/txts"
txt_files = os.listdir(path)
print(len(txt_files))

2100


# Putting Files in the Correct Places

In [ ]:
import random
import os
import shutil

path = "/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/txts"
files = os.listdir(path)

random.shuffle(files)

seventy_split = int(len(files) * 0.7)
fifteen_split = int(len(files) * 0.15)

for i in range(seventy_split):
  shutil.copy('/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/txts/{}'.format(files[i]), '/content/gdrive/MyDrive/aimi_summer_internship/yolo/labels/train/{}'.format(files[i]))
  shutil.copy('/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/jpgs/{}'.format(files[i][:-4] + ".jpg"), '/content/gdrive/MyDrive/aimi_summer_internship/yolo/images/train/{}'.format(files[i][:-4] + ".jpg"))

for i in range(fifteen_split):
  shutil.copy('/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/txts/{}'.format(files[i + seventy_split]), '/content/gdrive/MyDrive/aimi_summer_internship/yolo/labels/val/{}'.format(files[i + seventy_split]))
  shutil.copy('/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/jpgs/{}'.format(files[i + seventy_split][:-4] + ".jpg"), '/content/gdrive/MyDrive/aimi_summer_internship/yolo/images/val/{}'.format(files[i + seventy_split][:-4] + ".jpg"))

for i in range(fifteen_split):
  shutil.copy('/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/txts/{}'.format(files[i + seventy_split]), '/content/gdrive/MyDrive/aimi_summer_internship/yolo/labels/test/{}'.format(files[i + seventy_split]))
  shutil.copy('/content/gdrive/MyDrive/aimi_summer_internship/Dataset_Student/jpgs/{}'.format(files[i + seventy_split][:-4] + ".jpg"), '/content/gdrive/MyDrive/aimi_summer_internship/yolo/images/test/{}'.format(files[i + seventy_split][:-4] + ".jpg"))

In [ ]:
# check to make sure your files are of equal length

path = "/content/gdrive/MyDrive/aimi_summer_internship/yolo/images/train"
files = os.listdir(path)
print(len(files))
path = "/content/gdrive/MyDrive/aimi_summer_internship/yolo/images/val"
files = os.listdir(path)
print(len(files))
path = "/content/gdrive/MyDrive/aimi_summer_internship/yolo/images/test"
files = os.listdir(path)
print(len(files))

path = "/content/gdrive/MyDrive/aimi_summer_internship/yolo/labels/train"
files = os.listdir(path)
print(len(files))
path = "/content/gdrive/MyDrive/aimi_summer_internship/yolo/labels/val"
files = os.listdir(path)
print(len(files))
path = "/content/gdrive/MyDrive/aimi_summer_internship/yolo/labels/test"
files = os.listdir(path)
print(len(files))


1470
315
315
1470
315
315


#Setting up YOLO

This step assumes that your data is in the correct format + location.

In [ ]:
%cd /content/drive/MyDrive/aimi_summer_internship/yolo
# !git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
# !pip install -r requirements.txt  # install
%ls

/content/drive/MyDrive/aimi_summer_internship/yolo
/content/drive/MyDrive/aimi_summer_internship/yolo/yolov5
CONTRIBUTING.md  hubconf.py    README.md         train.py        yolov5s.pt
data/            LICENSE       requirements.txt  tutorial.ipynb
detect.py        models/       runs/             utils/
export.py        __pycache__/  setup.cfg         val.py


In [ ]:
# %%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# # parameters
# nc: {num_classes}  # number of classes
# depth_multiple: 0.33  # model depth multiple
# width_multiple: 0.50  # layer channel multiple

# # anchors
# anchors:
#   - [10,13, 16,30, 33,23]  # P3/8
#   - [30,61, 62,45, 59,119]  # P4/16
#   - [116,90, 156,198, 373,326]  # P5/32

# # YOLOv5 backbone
# backbone:
#   # [from, number, module, args]
#   [[-1, 1, Focus, [64, 3]],  # 0-P1/2
#    [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
#    [-1, 3, BottleneckCSP, [128]],
#    [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
#    [-1, 9, BottleneckCSP, [256]],
#    [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
#    [-1, 9, BottleneckCSP, [512]],
#    [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
#    [-1, 1, SPP, [1024, [5, 9, 13]]],
#    [-1, 3, BottleneckCSP, [1024, False]],  # 9
#   ]

# # YOLOv5 head
# head:
#   [[-1, 1, Conv, [512, 1, 1]],
#    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
#    [[-1, 6], 1, Concat, [1]],  # cat backbone P4
#    [-1, 3, BottleneckCSP, [512, False]],  # 13

#    [-1, 1, Conv, [256, 1, 1]],
#    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
#    [[-1, 4], 1, Concat, [1]],  # cat backbone P3
#    [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

#    [-1, 1, Conv, [256, 3, 2]],
#    [[-1, 14], 1, Concat, [1]],  # cat head P4
#    [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

#    [-1, 1, Conv, [512, 3, 2]],
#    [[-1, 10], 1, Concat, [1]],  # cat head P5
#    [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

#    [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
#   ]

In [ ]:
!python train.py --img 640 --batch 16 --epochs 300 --data medical_devices.yaml --weights yolov5s.pt

In [ ]:
!python val.py --img 640 --data medical_devices.yaml --weights /content/drive/MyDrive/runs/train/exp/weights/last.pt

val: data=/content/drive/MyDrive/aimi_summer_internship/yolo/yolov5/data/medical_devices.yaml, weights=['/content/drive/MyDrive/runs/train/exp/weights/last.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-266-g34df503 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7018216 parameters, 0 gradients
100% 755k/755k [00:00<00:00, 96.0MB/s]
val: Scanning '/content/drive/MyDrive/aimi_summer_internship/yolo/labels/val' images and labels...315 found, 0 missing, 203 empty, 0 corrupt: 100% 315/315 [00:25<00:00, 12.31it/s]
val: New cache created: /content/drive/MyDrive/aimi_summer_internship/yolo/labels/val.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:  